In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
gene_x_sample = pd.read_csv(PATH["gene_x_sample.processed.tsv"], sep="\t", index_col=0)

target_x_sample = pd.read_csv(PATH["target_x_sample.tsv"], sep="\t", index_col=0)

In [ ]:
def compute_log_ratio(logged_values, target):

    return logged_values[target == 1].mean() - logged_values[target == 0].mean()

In [ ]:
make_match_panel_keyword_arguments = dict(
    n_job=SETTING["n_job"],
    n_sampling=SETTING["n_sampling"],
    n_permutation=SETTING["n_permutation"],
    target_type="binary",
    plot_std=SETTING["plot_std"],
)

for target_name, target_values in target_x_sample.iterrows():

    output_directory_path = "{}/{}".format(
        PATH["differentially_expressed_gene/"], target_name
    )

    ccal.establish_path(output_directory_path, "directory")

    target_values = target_values[target_values != -1]

    score_moe_p_value_fdr = ccal.make_match_panel(
        target_values,
        gene_x_sample,
        n_extreme=SETTING["n_extreme"],
        title="All",
        file_path_prefix="{}/all".format(output_directory_path),
        **make_match_panel_keyword_arguments,
    )

    ccal.make_match_panel(
        target_values,
        gene_x_sample.reindex(SETTING["genes_to_peek"]),
        score_moe_p_value_fdr=score_moe_p_value_fdr,
        n_extreme=None,
        title="Peek",
        file_path_prefix="{}/peek".format(output_directory_path),
        **make_match_panel_keyword_arguments,
    )

    score_moe_p_value_fdr__log_ratio = ccal.make_match_panel(
        target_values,
        gene_x_sample,
        match_function=compute_log_ratio,
        n_extreme=SETTING["n_extreme"],
        title="All (Log Ratio)",
        file_path_prefix="{}/all.log_ratio".format(output_directory_path),
        **make_match_panel_keyword_arguments,
    )

    ccal.make_match_panel(
        target_values,
        gene_x_sample.reindex(SETTING["genes_to_peek"]),
        score_moe_p_value_fdr=score_moe_p_value_fdr__log_ratio,
        n_extreme=None,
        title="Peek (Log Ratio)",
        file_path_prefix="{}/peek.log_ratio".format(output_directory_path),
        **make_match_panel_keyword_arguments,
    )